In [ ]:
# import
import numpy as np
from keras.models import Model, load_model
from math import inf, floor
from statistics import mean
from matplotlib import pyplot as plt
from librosa import load
from librosa.effects import split
from librosa.feature import melspectrogram
from IPython import display as Ipd
from pyloudnorm import Meter as loudMeter
from pyloudnorm.normalize import loudness as loudnorm

In [ ]:
# constant
import os
import sys
parent_dir = os.path.abspath(os.path.join(os.getcwd(), "..\\00_Constant"))
if sys.path.count(parent_dir) == 0:
    sys.path.append(parent_dir)
import fyp_constants as constants

DATAFOLDER_PATH             = constants.DATA_FOLDER_PATH
MODELFOLDER_PATH            = constants.MODEL_FOLDER_PATH
EXTERNALDATA_FOLDER_PATH    = constants.EXTERNALTEST_DATA_FOLDER_PATH
DATA_PROCESSED_FOLDER_NM    = constants.DATA_PROCESSED_FOLDER_NM
FIGRUE_FOLDER_PATH          = constants.FIGRUE_FOLDER_PATH

EXTERNALDATA_CSV_PATH   = constants.EXTERNALTEST_DATA_CSVFILE_PATH

DATAX_MEL64_NM      = constants.DATAX_MEL64_NM
DATAX_MEL128_NM     = constants.DATAX_MEL128_NM
DATAX_MEL256_NM     = constants.DATAX_MEL256_NM
DATAX_WAVE_NM       = constants.DATAX_WAVE_NM
DATAX_WAVE_HALF_NM  = constants.DATAX_WAVE_HALF_NM

DATAY_NO_PROC_NM            = constants.DATAY_NO_PROC_NM
DATAY_NO_PROC_HALF_NM       = constants.DATAY_NO_PROC_HALF_NM
DATAY_PROC_VIB_NM           = constants.DATAY_PROC_VIB_NM
DATAY_PROC_VIB_GEN_NM       = constants.DATAY_PROC_VIB_GEN_NM
DATAY_PROC_VIB_GEN_BRE_NM   = constants.DATAY_PROC_VIB_GEN_BRE_NM
DATAY_PROC_VIB_GEN_BRI_NM   = constants.DATAY_PROC_VIB_GEN_BRI_NM
DATAY_PROC_VIB_GEN_GWL_NM   = constants.DATAY_PROC_VIB_GEN_GWL_NM
DATAY_PROC_GEN_MINMAXNORM   = constants.DATAY_PROC_GEN_MINMAXNORM

SAMPLING_RATE       = constants.SAMPLING_RATE
PARAMETERTYPE_LS    = constants.PARAMETERTYPE_LS
LOUDNORM_LUFS       = constants.LOUDNORM_LUFS
MEL_MAX_FREQ        = constants.MEL_MAX_FREQ

EXTERNAL_TESTING_SIZE = constants.EXTERNAL_TESTING_SIZE

In [ ]:
def loadData(dataset:int, dataX_nm:str, dataY_nm:str="data_y"):
    PROCESSDATA_FOLDER_PATH = f"{DATAFOLDER_PATH}\\{dataset}\\{DATA_PROCESSED_FOLDER_NM}"
    x = np.load(f"{PROCESSDATA_FOLDER_PATH}\\{dataX_nm}.npz")["data"]
    y = np.load(f"{PROCESSDATA_FOLDER_PATH}\\{dataY_nm}.npz")["data"]
    return (x,y)

# summary the predict to a dataset by mean, maximum predicted value and minmum predicted value for all 7 parameters
def summary_predict(predict_y):
    for idx, para in enumerate(PARAMETERTYPE_LS):
        predict_para = predict_y[:,idx]
        
        # mean
        para_mean = mean(predict_para)
        
        # max
        max_para = max(predict_para)
        max_para_idx = [idx for idx, err in enumerate(predict_para) if err == max_para]
        
        # min
        min_para = min(predict_para)
        min_para_idx = [idx for idx, err in enumerate(predict_para) if err == min_para]
        
        print(f"\t{para}\t{para_mean}\tmax {max_para}{max_para_idx}\tmin {min_para}{min_para_idx}")
        

In [ ]:
# select model to be tested
# model = load_model(f"{MODELFOLDER_PATH}/model1b/150.h5")
# model = load_model(f"{MODELFOLDER_PATH}/model2b1/425.h5")
# model = load_model(f"{MODELFOLDER_PATH}/model2b2/300.h5")
# model = load_model(f"{MODELFOLDER_PATH}/model2b3/300.h5")
# model = load_model(f"{MODELFOLDER_PATH}/model2c1/325.h5")
# model = load_model(f"{MODELFOLDER_PATH}/model2c2/450.h5")
# model = load_model(f"{MODELFOLDER_PATH}/model2c3/300.h5")
# model = load_model(f"{MODELFOLDER_PATH}/model2d1/275.h5")
# model = load_model(f"{MODELFOLDER_PATH}/model2d2/275.h5")
# model = load_model(f"{MODELFOLDER_PATH}/model2d3/275.h5")
# model = load_model(f"{MODELFOLDER_PATH}/model2e1/275.h5")
# model = load_model(f"{MODELFOLDER_PATH}/model2e2/475.h5")
# model = load_model(f"{MODELFOLDER_PATH}/model2e3/300.h5")
model = load_model(f"{MODELFOLDER_PATH}/model3_1/200.h5")
# model = load_model(f"{MODELFOLDER_PATH}/model3_2/100.h5")
# model = load_model(f"{MODELFOLDER_PATH}/model3_3/150.h5")

Internal test

In [ ]:
# select internal testing set
# (x,y) = loadData(dataset="3", dataX_nm=DATAX_MEL64_NM, dataY_nm=DATAY_NO_PROC_NM)
# (x,y) = loadData(dataset="3", dataX_nm=DATAX_MEL128_NM, dataY_nm=DATAY_PROC_VIB_NM)
# (x,y) = loadData(dataset="3", dataX_nm=DATAX_MEL128_NM, dataY_nm=DATAY_PROC_VIB_GEN_NM)
# (x,y) = loadData(dataset="3", dataX_nm=DATAX_MEL128_NM, dataY_nm=DATAY_PROC_VIB_GEN_BRE_NM)
# (x,y) = loadData(dataset="3", dataX_nm=DATAX_MEL128_NM, dataY_nm=DATAY_PROC_VIB_GEN_BRI_NM)
# (x,y) = loadData(dataset="3", dataX_nm=DATAX_MEL128_NM, dataY_nm=DATAY_PROC_VIB_GEN_GWL_NM)
(x,y) = loadData(dataset="6", dataX_nm=DATAX_MEL128_NM, dataY_nm=DATAY_PROC_GEN_MINMAXNORM)

In [ ]:
# acceptable error
# ERROR_THESHOLD = [10,16,16,16,16,26,16]

# the acceptable error theshold after min max norm
ERROR_THESHOLD = [13,18,48,20,16,26,16]

In [ ]:
# estimate the model accuracy by the acceptable error
def accuracy(predict_y, true_y):
    CNT_RECORD = len(predict_y)
    
    err_y = abs(predict_y-true_y)
    
    for idx, para in enumerate(PARAMETERTYPE_LS):
        para_err_y = err_y[:,idx]
        
        # cnt para has error that over theashold
        cnt_err = 0
        for err in para_err_y:
            if err > ERROR_THESHOLD[idx]:
                cnt_err += 1
        accuracy = round((CNT_RECORD-cnt_err)/CNT_RECORD,4)
        
        # max err
        max_para_err = max(para_err_y)
        max_para_err_idx = [idx for idx, err in enumerate(para_err_y) if err == max_para_err]
        
        # min err
        min_para_err = min(para_err_y)
        min_para_err_idx = [idx for idx, err in enumerate(para_err_y) if err == min_para_err]
        
        print(f"\t{para}\t{accuracy}\tmax err {round(max_para_err,4)}{max_para_err_idx}\tmin err {round(min_para_err,4)}{min_para_err_idx}")

# plot a historgram for checking the predicted value distribution
def plotDistribution(y):
    for idx, para, in enumerate(PARAMETERTYPE_LS):
        print(para)
        plt.hist(y[:,idx], color = 'blue',bins = 128)
        plt.xlim(-20,220)
        plt.show()

In [ ]:
# predict the testing data
y_overall = model.predict(x)
# predict result of each vocal database
y_V01 = y_overall[0:1000]
y_V02 = y_overall[1000:2000]
y_V03 = y_overall[2000:3000]
y_V04 = y_overall[3000:4000]
y_V05 = y_overall[4000:5000]
y_V06 = y_overall[5000:6000]

In [ ]:
# mean, max value, min value of 7 parameter
print("Overall")
summary_predict(y_overall)
print("V01")
summary_predict(y_V01)
print("V02")
summary_predict(y_V02)
print("V03")
summary_predict(y_V03)
print("V04")
summary_predict(y_V04)
print("V05")
summary_predict(y_V05)
print("V06")
summary_predict(y_V06)

In [ ]:
# accuracy, using acceptable error
print("Overall")
accuracy(y_overall, y)
print("V01")
accuracy(y_V01, y[0:1000])
print("V02")
accuracy(y_V02, y[1000:2000])
print("V03")
accuracy(y_V03, y[2000:3000])
print("V04")
accuracy(y_V04, y[3000:4000])
print("V05")
accuracy(y_V05, y[4000:5000])
print("V06")
accuracy(y_V06, y[5000:6000])

In [ ]:
# distribution of true y
plotDistribution(y)

In [ ]:
# distribution of predict y
plotDistribution(y_overall)

External Test

In [ ]:
# functions
# plot a box plot for predict y
def plotBox(predict_y, save_nm:None):
    data = [predict_y[:,idx] for idx in range(len(PARAMETERTYPE_LS))]
    
    fig = plt.figure()
    ax = fig.add_subplot()
    ax.set_xticklabels(["BRE","BRI","CLE","GEN","GWL","OPE","Vib"])
    
    plt.ylim(-20,220)
    plt.boxplot(data)
    
    if save_nm is None:
        plt.show()
    else:
        plt.savefig(save_nm)

# comparing mean of parameter predicted values between two external testing
def compare_external_test(y1, y2, y1_nm="", y2_nm=""):
    y1_para = [0 for _ in range(len(PARAMETERTYPE_LS))]
    y2_para = [0 for _ in range(len(PARAMETERTYPE_LS))]
    
    for record in y1:
        for idx in range(len(record)):
            y1_para[idx] += record[idx]
            
    for record in y2:
        for idx in range(len(record)):
            y2_para[idx] += record[idx]
            
    
    print(f"[{y1_nm} - {y2_nm}]")
    for idx in range(len(y1_para)):
        print(f"\t{PARAMETERTYPE_LS[idx]}\t{y1_para[idx]/len(y1) - y2_para[idx]/len(y2)}")

# list the index of record that obtains top 10 highest/10 lowest value for each parameters
def show10MinMax(y_predict):
    # append idx
    y = [[*record, idx+1] for idx, record in enumerate(y_predict)] 
    
    for idx, para in enumerate(PARAMETERTYPE_LS):
        y_sort = sorted(y, key=lambda record:record[idx])

        min10 = np.array(y_sort[:10])[:,idx]
        min10_idx = np.array(y_sort[:10])[:,-1]

        max10 = y_sort[-10:]
        max10.reverse()
        max10 = np.array(max10)[:,idx]
        
        max10_idx = y_sort[-10:]
        max10_idx.reverse()
        max10_idx = np.array(max10_idx)[:,-1]
        
        print(para)
        print(f"min10 {min10_idx}\tmax10 {max10_idx}")
        print(f"      {min10}\t      {max10}")

In [ ]:
# process external samples
external_x = []

# 0 for mel 64
# 1 for mel 128
# 2 for mel 256
# 3 for waveform
x_type = 1

for idx in range(EXTERNAL_TESTING_SIZE):
    audio, sr = load(f"{EXTERNALDATA_FOLDER_PATH}\\{idx+1}.wav", sr=SAMPLING_RATE)
    
    # the length of the record is supposed to be >= 4 seconds
    if(len(audio)/sr < 4):
        print(f"audio {idx+1} length less then 4s")
    
    # clip first 4 second non slience part of the recording
    interval = split(y=audio, top_db=50)
    audio = audio[interval[0][0]:interval[0][0]+sr*4]
    
    # loudness normalization
    loudness = loudMeter(sr).integrated_loudness(audio)
    audio_norm = loudnorm(audio, loudness, LOUDNORM_LUFS)
        
    if x_type == 0:
        data_x = melspectrogram(y=audio, sr=sr, n_mels=64, fmax=MEL_MAX_FREQ)
    elif x_type == 1:
        data_x = melspectrogram(y=audio, sr=sr, n_mels=128, fmax=MEL_MAX_FREQ)
    elif x_type == 2:
        data_x = melspectrogram(y=audio, sr=sr, n_mels=256, fmax=MEL_MAX_FREQ)
    elif x_type == 3:
        data_x = audio
        
    external_x.append(data_x)

In [ ]:
# predict all the external test recording
all_predict = model.predict(np.array(external_x))

# helper function, query the prediction result by list of index, refer to the external.csv for the index
def getPredict(idx_ls):
    return np.array([predict for idx, predict in enumerate(all_predict) if idx+1 in idx_ls])

In [ ]:
# All
print("[all]")
summary_predict(all_predict)
plotBox(all_predict, save_nm=f"{FIGRUE_FOLDER_PATH}/00all")
show10MinMax(all_predict)

In [ ]:
# index list of different sets in External Test

# VocalSet
data = np.loadtxt(EXTERNALDATA_CSV_PATH, delimiter=",", dtype=str)
data = np.array(data[1:,0:-3]).astype(int)

COLUMN_BREATHY  = 1
COLUMN_BELT     = 2
COLUMN_GENDER   = 3
COLUMN_FRY      = 4
COLUMN_VIBRATE  = 5

breathy = data[data[:,COLUMN_BREATHY] == 1][:,0]
belt    = data[data[:,COLUMN_BELT] == 1][:,0]
fry     = data[data[:,COLUMN_FRY] == 1][:,0]
vibrate = data[data[:,COLUMN_VIBRATE] == 1][:,0]
straight  = data[(data[:,COLUMN_BREATHY] == 0) & (data[:,COLUMN_BELT] == 0) & (data[:,COLUMN_FRY] == 0) & (data[:,COLUMN_VIBRATE] == 0)][:,0]

# CVT
CVT_breathy         = [520,521,522,523,524,525]
CVT_no_vibrate      = [529,535]
CVT_fast_vibrate    = [526,532]
CVT_hammer_vibrate  = [527,528,533,534]
CVT_normal_vibrate  = [530,536]
CVT_slow_vibrate    = [531]
CVT_scream          = [537,538,539,540,541,542]
CVT_distortion      = [543,544,545,546,547]
CVT_growl           = [548,549,550,551,552,553,554,555,556,557]
CVT_grunt           = [558,559]

# YT
YT_breathy                  = [411,412,413,414,415,416]
YT_breathy_compare          = [414]
YT_non_breathy_compare      = [515]
YT_bright                   = [516]
YT_dark                     = [517]
YT_good_bright              = [518]
YT_bad_bright               = [519]
YT_mouth_open_1_horizon     = [560,561]
YT_mouth_open_1_horizonL    = [562]
YT_mouth_open_1_vertical    = [563,564,565,566]
YT_mouth_open_2_close       = [567,568]
YT_mouth_open_2_open        = [569,570,571]
YT_mouth_open_3_tooSmall    = [572]
YT_mouth_open_3_small       = [573]
YT_mouth_open_3_normal      = [574]
YT_mouth_open_3_wide        = [575]
YT_mouth_open_3_big         = [576]
YT_mouth_open_3_tooLong     = [577]

# gender
female  = data[data[:,COLUMN_GENDER] == 0][:,0]
male    = data[data[:,COLUMN_GENDER] == 1][:,0]
child   = data[data[:,COLUMN_GENDER] == 2][:,0]

External Test: VocalSet

In [ ]:
y1 = getPredict(breathy)
y2 = getPredict([*vibrate, *straight, *fry, *belt])
print("[breathy]")
summary_predict(y1)
plotBox(y1, save_nm=f"{FIGRUE_FOLDER_PATH}/01breathy")
print("[non breathy]")
summary_predict(y2)
plotBox(y2, save_nm=f"{FIGRUE_FOLDER_PATH}/02non_breathy")
compare_external_test(y1,y2, "breathy", "non breathy")

In [ ]:
y1 = getPredict(belt)
y2 = getPredict([*vibrate, *straight, *fry, *breathy])
print("[belt]")
summary_predict(y1)
plotBox(y1, save_nm=f"{FIGRUE_FOLDER_PATH}/03bright")
print("[non belt]")
summary_predict(y2)
plotBox(y2, save_nm=f"{FIGRUE_FOLDER_PATH}/04non_bright")
compare_external_test(y1,y2, "bright", "non belt")

In [ ]:
y1 = getPredict(vibrate)
y2 = getPredict([*fry, *straight, *breathy, *belt])
print("[vibrate]")
summary_predict(y1)
plotBox(y1, save_nm=f"{FIGRUE_FOLDER_PATH}/05vibrate")
print("[non vibrate]")
summary_predict(y2)
plotBox(y2, save_nm=f"{FIGRUE_FOLDER_PATH}/06non_vibrate")
compare_external_test(y1,y2, "vibrate", "non vibrate")

In [ ]:
y1 = getPredict(fry)
y2 = getPredict([*straight, *vibrate, *breathy, *belt])
print("[fry]")
summary_predict(y1)
plotBox(y1, save_nm=f"{FIGRUE_FOLDER_PATH}/07fry")
print("[non fry]")
summary_predict(y2)
plotBox(y2, save_nm=f"{FIGRUE_FOLDER_PATH}/08non_fry")
compare_external_test(y1,y2, "growl", "non fry")

CVT

In [ ]:
# CVT breathy
y1 = getPredict(CVT_breathy)
print("[CVT breathy]")
summary_predict(y1)
plotBox(y1, save_nm=f"{FIGRUE_FOLDER_PATH}/09CVT_breathy")

In [ ]:
# CVT vibrate
print("[CVT no]")
y = getPredict(CVT_no_vibrate)
summary_predict(y)
plotBox(y, save_nm=f"{FIGRUE_FOLDER_PATH}/10CVT_no_vibrate")

print("[CVT fast]")
y = getPredict(CVT_fast_vibrate)
summary_predict(y)
plotBox(y, save_nm=f"{FIGRUE_FOLDER_PATH}/11CVT_fast_vibrate")

print("[CVT hammer]")
y = getPredict(CVT_hammer_vibrate)
summary_predict(y)
plotBox(y, save_nm=f"{FIGRUE_FOLDER_PATH}/12CVT_hammer_vibrate")

print("[CVT normal_v]")
y = getPredict(CVT_normal_vibrate)
summary_predict(y)
plotBox(y, save_nm=f"{FIGRUE_FOLDER_PATH}/13CVT_normal_vibrate")

print("[CVT slow]")
y = getPredict(CVT_slow_vibrate)
summary_predict(y)
plotBox(y, save_nm=f"{FIGRUE_FOLDER_PATH}/14CVT_slow_vibrate")

In [ ]:
# CVT scream
y1 = getPredict(CVT_scream)
print("[CVT scream]")
summary_predict(y1)
plotBox(y1, save_nm=f"{FIGRUE_FOLDER_PATH}/15CVT_scream")

In [ ]:
# CVT distortion
y1 = getPredict(CVT_distortion)
print("[CVT distortion]")
summary_predict(y1)
plotBox(y1, save_nm=f"{FIGRUE_FOLDER_PATH}/16CVT_distortion")

In [ ]:
# CVT growl
y1 = getPredict(CVT_growl)
print("[CVT growl]")
summary_predict(y1)
plotBox(y1, save_nm=f"{FIGRUE_FOLDER_PATH}/17CVT_growl")

In [ ]:
# CVT grunt
y1 = getPredict(CVT_grunt)
print("[CVT grunt]")
summary_predict(y1)
plotBox(y1, save_nm=f"{FIGRUE_FOLDER_PATH}/18CVT_grunt")

YT

In [ ]:
# YT breathy
y1 = getPredict(YT_breathy)
print("[YT_breathy]")
summary_predict(y1)
plotBox(y1, save_nm=f"{FIGRUE_FOLDER_PATH}/19YT_breathy")

In [ ]:
# YT breathy (compare breathy and non breathy)
y1 = getPredict(YT_breathy_compare)
y2 = getPredict(YT_non_breathy_compare)
print("[breathy]")
summary_predict(y1)
plotBox(y1, save_nm=f"{FIGRUE_FOLDER_PATH}/20YT_breathy_compare")
print("[non breathy]")
summary_predict(y2)
plotBox(y2, save_nm=f"{FIGRUE_FOLDER_PATH}/21YT_non_breathy_compare")
compare_external_test(y1,y2, "breathy", "non breathy")

In [ ]:
# YT bright
y1 = getPredict(YT_bright)
y2 = getPredict(YT_dark)
print("[516 bright]")
summary_predict(y1)
plotBox(y1, save_nm=f"{FIGRUE_FOLDER_PATH}/22YT_bright")
print("[517 dark]")
summary_predict(y2)
plotBox(y2, save_nm=f"{FIGRUE_FOLDER_PATH}/23YT_dark")
compare_external_test(y1,y2, "516", "517")

In [ ]:
# YT bright
y1 = getPredict(YT_good_bright)
y2 = getPredict(YT_bad_bright)
print("[518 bright]")
summary_predict(y1)
plotBox(y1, save_nm=f"{FIGRUE_FOLDER_PATH}/24YT_good_bright")
print("[519 bad bright]")
summary_predict(y2)
plotBox(y2, save_nm=f"{FIGRUE_FOLDER_PATH}/25YT_bad_bright")
compare_external_test(y1,y2, "518", "519")

In [ ]:
# YT mouth opening 1
y1 = getPredict(YT_mouth_open_1_horizon)
y2 = getPredict(YT_mouth_open_1_horizonL)
y3 = getPredict(YT_mouth_open_1_vertical)

print("[horizon]")
summary_predict(y1)
plotBox(y1, save_nm=f"{FIGRUE_FOLDER_PATH}/26YT_mouth_open_1_horizon")

print("[large horizon]")
summary_predict(y2)
plotBox(y2, save_nm=f"{FIGRUE_FOLDER_PATH}/27YT_mouth_open_1_horizonL")

print("[vertical]")
summary_predict(y2)
plotBox(y2, save_nm=f"{FIGRUE_FOLDER_PATH}/28YT_mouth_open_1_vertical")

In [ ]:
# YT mouth opening 2
y1 = getPredict(YT_mouth_open_2_close)
y2 = getPredict(YT_mouth_open_2_open)

print("[close]")
summary_predict(y1)
plotBox(y1, save_nm=f"{FIGRUE_FOLDER_PATH}/29YT_mouth_open_2_close")

print("[open]")
summary_predict(y2)
plotBox(y2, save_nm=f"{FIGRUE_FOLDER_PATH}/30YT_mouth_open_2_open")

In [ ]:
# YT mouth opening 3
y1 = getPredict(YT_mouth_open_3_tooSmall)
y2 = getPredict(YT_mouth_open_3_small)
y3 = getPredict(YT_mouth_open_3_normal)
y4 = getPredict(YT_mouth_open_3_wide)
y5 = getPredict(YT_mouth_open_3_big)
y6 = getPredict(YT_mouth_open_3_tooLong)

print("[too small]")
summary_predict(y1)
plotBox(y1, save_nm=f"{FIGRUE_FOLDER_PATH}/31YT_mouth_open_3_tooSmall")

print("[small]")
summary_predict(y2)
plotBox(y2, save_nm=f"{FIGRUE_FOLDER_PATH}/32YT_mouth_open_3_small")

print("[normal]")
summary_predict(y3)
plotBox(y3, save_nm=f"{FIGRUE_FOLDER_PATH}/33YT_mouth_open_3_normal")

print("[wide]")
summary_predict(y4)
plotBox(y4, save_nm=f"{FIGRUE_FOLDER_PATH}/34YT_mouth_open_3_wide")

print("[big]")
summary_predict(y5)
plotBox(y5, save_nm=f"{FIGRUE_FOLDER_PATH}/35YT_mouth_open_3_big")

print("[too long]")
summary_predict(y6)
plotBox(y6, save_nm=f"{FIGRUE_FOLDER_PATH}/36YT_mouth_open_3_tooLong")

Gender

In [ ]:
y1 = getPredict(child)
y2 = getPredict(male)
y3 = getPredict(female)
print("[child]")
summary_predict(y1)
plotBox(y1, save_nm=f"{FIGRUE_FOLDER_PATH}/37child")
print("[male]")
summary_predict(y2)
plotBox(y2, save_nm=f"{FIGRUE_FOLDER_PATH}/38male")
print("[female]")
summary_predict(y3)
plotBox(y3, save_nm=f"{FIGRUE_FOLDER_PATH}/39female")
compare_external_test(y1,y2, "child", "male")
compare_external_test(y1,y3, "child", "female")